# Project 3 : Web APIs & NLP - Classification of subreddit posts

## Part 1 - Pre-scraping data exploration

## Import Libraries

In [1]:
# import libraries

import requests
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

In [2]:
# set config to display all
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

## Data scraping from Reddit via pushshift.api - test case

The two chosen subreddits are "r/investing" and "r/CryptoCurrency".\
Start with a sample trial of scraping from the subreddit of "investing".\
This step is to get acquainted with the data prior to a full scraping of all the requisite data.\
By having some sense of the data attributes, i.e. columns type and information, I could perhaps scrape a very selective and specific parts of the data instead of brute force scraping of the entire dataset, which could take up more time.

In [3]:
# do a sample trial of scraping from reddit
# define url
url = 'https://api.pushshift.io/reddit/search/submission'

# define parameters
# setting before 25 Oct 2021 to ensure the 100 results are reproducible and do not change with time
# 25 Oct 2021 2359hrs, SG time is 1633017600, reference from https://epochconverter.com
params = {
    'subreddit':'investing',
    'size':100,
    'before':1633017600
}

# set the response
response = requests.get(url, params)

In [4]:
# check the response status, 200 means ok
response.status_code

200

In [5]:
# convert the JSON string into dictionary
invest_data = response.json()
# preview the data
invest_data

{'data': [{'all_awardings': [],
   'allow_live_comments': False,
   'author': 'mattlas',
   'author_flair_css_class': None,
   'author_flair_richtext': [],
   'author_flair_text': None,
   'author_flair_type': 'text',
   'author_fullname': 't2_9gebh',
   'author_is_blocked': False,
   'author_patreon_flair': False,
   'author_premium': False,
   'awarders': [],
   'can_mod_post': False,
   'contest_mode': False,
   'created_utc': 1633017223,
   'domain': 'self.investing',
   'full_link': 'https://www.reddit.com/r/investing/comments/pymndc/ama_with_caias_ceo_bill_kelly_and_exec_vp_john/',
   'gildings': {},
   'id': 'pymndc',
   'is_created_from_ads_ui': False,
   'is_crosspostable': False,
   'is_meta': False,
   'is_original_content': False,
   'is_reddit_media_domain': False,
   'is_robot_indexable': False,
   'is_self': True,
   'is_video': False,
   'link_flair_background_color': '',
   'link_flair_richtext': [],
   'link_flair_text_color': 'dark',
   'link_flair_type': 'text',
   

In [6]:
# extract the list of posts from the dictionary
invest_posts = invest_data['data']

# check number of posts, 100 is right
print(len(invest_posts))

# preview the first post
invest_posts[0]

100


{'all_awardings': [],
 'allow_live_comments': False,
 'author': 'mattlas',
 'author_flair_css_class': None,
 'author_flair_richtext': [],
 'author_flair_text': None,
 'author_flair_type': 'text',
 'author_fullname': 't2_9gebh',
 'author_is_blocked': False,
 'author_patreon_flair': False,
 'author_premium': False,
 'awarders': [],
 'can_mod_post': False,
 'contest_mode': False,
 'created_utc': 1633017223,
 'domain': 'self.investing',
 'full_link': 'https://www.reddit.com/r/investing/comments/pymndc/ama_with_caias_ceo_bill_kelly_and_exec_vp_john/',
 'gildings': {},
 'id': 'pymndc',
 'is_created_from_ads_ui': False,
 'is_crosspostable': False,
 'is_meta': False,
 'is_original_content': False,
 'is_reddit_media_domain': False,
 'is_robot_indexable': False,
 'is_self': True,
 'is_video': False,
 'link_flair_background_color': '',
 'link_flair_richtext': [],
 'link_flair_text_color': 'dark',
 'link_flair_type': 'text',
 'locked': False,
 'media_only': False,
 'no_follow': True,
 'num_comment

In [7]:
# construct a dataframe for the posts
dftest = pd.DataFrame(invest_posts)
dftest.head()

,all_awardings,allow_live_comments,author,author_flair_css_class,author_flair_richtext,author_flair_text,author_flair_type,author_fullname,author_is_blocked,author_patreon_flair,author_premium,awarders,can_mod_post,contest_mode,created_utc,domain,full_link,gildings,id,is_created_from_ads_ui,is_crosspostable,is_meta,is_original_content,is_reddit_media_domain,is_robot_indexable,is_self,is_video,link_flair_background_color,link_flair_richtext,link_flair_text_color,link_flair_type,locked,media_only,no_follow,num_comments,num_crossposts,over_18,parent_whitelist_status,permalink,pinned,pwls,removed_by_category,retrieved_on,score,selftext,send_replies,spoiler,stickied,subreddit,subreddit_id,subreddit_subscribers,subreddit_type,thumbnail,title,total_awards_received,treatment_tags,upvote_ratio,url,whitelist_status,wls,post_hint,preview,suggested_sort
0,[],False,mattlas,None,[],None,text,t2_9gebh,False,False,False,[],False,False,1633017223,self.investing,https://www.reddit.com/r/investing/comments/py...,{},pymndc,False,False,False,False,False,False,True,False,,[],dark,text,False,False,True,2,0,False,all_ads,/r/investing/comments/pymndc/ama_with_caias_ce...,False,6,reddit,1633017234,1,[removed],True,False,False,investing,t5_2qhhq,1922293,public,self,AMA with CAIA's CEO Bill Kelly and Exec. VP Jo...,0,[],1.0,https://www.reddit.com/r/investing/comments/py...,all_ads,6,NaN,NaN,NaN
1,[],False,averymell0r,None,[],None,text,t2_6ljt0jh5,False,False,False,[],False,False,1633015463,self.investing,https://www.reddit.com/r/investing/comments/py...,{},pym22t,False,False,False,False,False,False,True,False,,[],dark,text,False,False,True,2,0,False,all_ads,/r/investing/comments/pym22t/sp500_others_wort...,False,6,moderator,1633015475,1,[removed],True,False,False,investing,t5_2qhhq,1922279,public,self,S&amp;P500 &amp; Others. Worth pulling out?,0,[],1.0,https://www.reddit.com/r/investing/comments/py...,all_ads,6,NaN,NaN,NaN
2,[],False,SolidFaiz,None,[],None,text,t2_dyuny7s,False,False,False,[],False,False,1633015283,self.investing,https://www.reddit.com/r/investing/comments/py...,{},pylzwi,False,False,False,False,False,False,True,False,,[],dark,text,False,False,True,2,0,False,all_ads,/r/investing/comments/pylzwi/deciding_which_ne...,False,6,moderator,1633015294,1,[removed],True,False,False,investing,t5_2qhhq,1922277,public,self,Deciding which news source to subscribe to,0,[],1.0,https://www.reddit.com/r/investing/comments/py...,all_ads,6,NaN,NaN,NaN
3,[],False,thrownawaypaperream,None,[],None,text,t2_bima2dn4,False,False,False,[],False,False,1633013003,self.investing,https://www.reddit.com/r/investing/comments/py...,{},pyl7zi,False,False,False,False,False,False,True,False,,[],dark,text,False,False,True,2,0,False,all_ads,/r/investing/comments/pyl7zi/i_want_to_get_int...,False,6,moderator,1633013014,1,[removed],True,False,False,investing,t5_2qhhq,1922259,public,self,I want to get into investing,0,[],1.0,https://www.reddit.com/r/investing/comments/py...,all_ads,6,NaN,NaN,NaN
4,[],False,Tech-Genius-780,None,[],None,text,t2_dq89htox,False,False,False,[],False,False,1633012201,self.investing,https://www.reddit.com/r/investing/comments/py...,{},pykydk,False,False,False,False,False,False,True,False,,[],dark,text,False,False,True,2,0,False,all_ads,/r/investing/comments/pykydk/what_are_your_tho...,False,6,moderator,1633012214,1,[removed],True,False,False,investing,t5_2qhhq,1922258,public,self,What are your thoughts on SPACs?,0,[],1.0,https://www.reddit.com/r/investing/comments/py...,all_ads,6,NaN,NaN,NaN


In [8]:
# take an exploratory look
dftest.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 63 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   all_awardings                100 non-null    object 
 1   allow_live_comments          100 non-null    bool   
 2   author                       100 non-null    object 
 3   author_flair_css_class       0 non-null      object 
 4   author_flair_richtext        100 non-null    object 
 5   author_flair_text            0 non-null      object 
 6   author_flair_type            100 non-null    object 
 7   author_fullname              100 non-null    object 
 8   author_is_blocked            100 non-null    bool   
 9   author_patreon_flair         100 non-null    bool   
 10  author_premium               100 non-null    bool   
 11  awarders                     100 non-null    object 
 12  can_mod_post                 100 non-null    bool   
 13  contest_mode         

The columns 'removed_by_category' catches my eye. Of these 100 results, does it mean that 77 of them are removed?. Take a deeper look

In [9]:
# take a look at the data in removed_by_category, so far 2 categories are observed : reddit, moderator
# looping up to change the timestamp for another 3 times and see if there are different categories
# observed another category : deleted
dftest['removed_by_category'].value_counts()

moderator    77
reddit        9
Name: removed_by_category, dtype: int64

In [10]:
# seeing a pattern here, seems like posts have 'removed_by_category' column
dftest[['selftext','removed_by_category', 'is_crosspostable', 
        'is_robot_indexable']][dftest['removed_by_category'].isnull()]

,selftext,removed_by_category,is_crosspostable,is_robot_indexable
9,"Hey Everyone,\n\nI wanted to share an analysis...",NaN,True,True
19,Raymond James analyst Steven Seedhouse raised ...,NaN,True,True
27,It seems like half the articles about the stoc...,NaN,True,True
29,Have a general question? Want to offer some c...,NaN,True,True
30,"If your question is ""I have $10,000, what do I...",NaN,True,True
39,"I am 24, make about ~$60k per year and it will...",NaN,True,True
42,I know this entirely depends on what I invest ...,NaN,True,True
44,\n\n* Ford to bring electric zero-emission ve...,NaN,True,True
54,"Howdy, \n\nAs a swing-trader, my favorite play...",NaN,True,True
67,My spouse has a rollover 401/ traditional IRA...,NaN,True,True


In [11]:
dftest[['selftext','removed_by_category', 'is_crosspostable', 
        'is_robot_indexable']][dftest['removed_by_category'].notnull()]

,selftext,removed_by_category,is_crosspostable,is_robot_indexable
0,[removed],reddit,False,False
1,[removed],moderator,False,False
2,[removed],moderator,False,False
3,[removed],moderator,False,False
4,[removed],moderator,False,False
5,[removed],moderator,False,False
6,[removed],moderator,False,False
7,[removed],moderator,False,False
8,[removed],moderator,False,False
10,[removed],moderator,False,False


Therefore, I could either use the 'is_crosspostable' or 'is_robot_indexable' columns to filter our the removed posts.

There are two thoughts on this, I could simply extract all the data first before doing data clean-up or I could simply omit these removed posts since I have already found out about them and reduce the data load for my first scraped dataset.\
This is also based on cross-checking with subreddit to confirm that these posts do not exist.
I have decided to omit these removed posts to keep the dataset smaller since I have already detected these removed posts. There could be other removed posts with a different category too that I might not have detected in this prelim testing. Those can be addressed at a later stage.

## Repeat the same process for the 2nd category - investing

In [12]:
# do a sample trial of scraping from reddit
# define url
url = 'https://api.pushshift.io/reddit/search/submission'

# define parameters
# setting before 25 Oct 2021 to ensure the 100 results are reproducible and do not change with time
# 25 Oct 2021 2359hrs, SG time is 1633017600, reference from https://epochconverter.com
params = {
    'subreddit':'CryptoCurrency',
    'size':100,
    'before':1633017600
}

# set the response
response = requests.get(url, params)

In [13]:
# check the response status, 200 means ok
response.status_code

200

In [14]:
# convert the JSON string into dictionary
crypto_data = response.json()
# preview the data
crypto_data

{'data': [{'all_awardings': [],
   'allow_live_comments': False,
   'author': 'Conversiongfx',
   'author_flair_background_color': '',
   'author_flair_css_class': 'Transitioning',
   'author_flair_richtext': [{'e': 'text', 't': ' '}],
   'author_flair_text': ' ',
   'author_flair_text_color': 'dark',
   'author_flair_type': 'richtext',
   'author_fullname': 't2_hq3cene',
   'author_is_blocked': False,
   'author_patreon_flair': False,
   'author_premium': False,
   'awarders': [],
   'can_mod_post': False,
   'contest_mode': False,
   'created_utc': 1633017594,
   'domain': 'self.CryptoCurrency',
   'full_link': 'https://www.reddit.com/r/CryptoCurrency/comments/pymrum/staking_yield_farming_in_crypto_for_beginners/',
   'gildings': {},
   'id': 'pymrum',
   'is_created_from_ads_ui': False,
   'is_crosspostable': True,
   'is_meta': False,
   'is_original_content': False,
   'is_reddit_media_domain': False,
   'is_robot_indexable': True,
   'is_self': True,
   'is_video': False,
   'lin

In [15]:
# extract the list of posts from the dictionary
crypto_posts = crypto_data['data']

# check number of posts, 100 is right
print(len(crypto_posts))

# preview the first post
crypto_posts[0]

100


{'all_awardings': [],
 'allow_live_comments': False,
 'author': 'Conversiongfx',
 'author_flair_background_color': '',
 'author_flair_css_class': 'Transitioning',
 'author_flair_richtext': [{'e': 'text', 't': ' '}],
 'author_flair_text': ' ',
 'author_flair_text_color': 'dark',
 'author_flair_type': 'richtext',
 'author_fullname': 't2_hq3cene',
 'author_is_blocked': False,
 'author_patreon_flair': False,
 'author_premium': False,
 'awarders': [],
 'can_mod_post': False,
 'contest_mode': False,
 'created_utc': 1633017594,
 'domain': 'self.CryptoCurrency',
 'full_link': 'https://www.reddit.com/r/CryptoCurrency/comments/pymrum/staking_yield_farming_in_crypto_for_beginners/',
 'gildings': {},
 'id': 'pymrum',
 'is_created_from_ads_ui': False,
 'is_crosspostable': True,
 'is_meta': False,
 'is_original_content': False,
 'is_reddit_media_domain': False,
 'is_robot_indexable': True,
 'is_self': True,
 'is_video': False,
 'link_flair_background_color': '',
 'link_flair_css_class': 'META',
 'li

In [16]:
# construct a dataframe for the posts
dftest = pd.DataFrame(crypto_posts)
dftest.head()

,all_awardings,allow_live_comments,author,author_flair_background_color,author_flair_css_class,author_flair_richtext,author_flair_text,author_flair_text_color,author_flair_type,author_fullname,author_is_blocked,author_patreon_flair,author_premium,awarders,can_mod_post,contest_mode,created_utc,domain,full_link,gildings,id,is_created_from_ads_ui,is_crosspostable,is_meta,is_original_content,is_reddit_media_domain,is_robot_indexable,is_self,is_video,link_flair_background_color,link_flair_css_class,link_flair_richtext,link_flair_text,link_flair_text_color,link_flair_type,locked,media_only,no_follow,num_comments,num_crossposts,over_18,parent_whitelist_status,permalink,pinned,post_hint,preview,pwls,retrieved_on,score,selftext,send_replies,spoiler,stickied,subreddit,subreddit_id,subreddit_subscribers,subreddit_type,thumbnail,title,total_awards_received,treatment_tags,upvote_ratio,url,whitelist_status,wls,link_flair_template_id,thumbnail_height,thumbnail_width,url_overridden_by_dest,removed_by_category,media,media_embed,secure_media,secure_media_embed,author_flair_template_id,crosspost_parent,crosspost_parent_list,discussion_type,suggested_sort
0,[],False,Conversiongfx,,Transitioning,"[{'e': 'text', 't': ' '}]",,dark,richtext,t2_hq3cene,False,False,False,[],False,False,1633017594,self.CryptoCurrency,https://www.reddit.com/r/CryptoCurrency/commen...,{},pymrum,False,True,False,False,False,True,True,False,,META,"[{'e': 'text', 't': 'META'}]",META,dark,richtext,False,False,True,10,0,False,all_ads,/r/CryptoCurrency/comments/pymrum/staking_yiel...,False,self,"{'enabled': False, 'images': [{'id': 'Ak7UcWZq...",6,1633017605,1,Thought the links in this post might be useful...,True,False,False,CryptoCurrency,t5_2wlj3,3520483,public,self,"Staking, Yield Farming in Crypto for beginners",0,[],1.0,https://www.reddit.com/r/CryptoCurrency/commen...,all_ads,6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,[],False,ADD-DDS,,None,"[{'e': 'text', 't': 'Gold | QC: CC 243'}]",Gold | QC: CC 243,dark,richtext,t2_51z728bj,False,False,False,[],False,False,1633017584,self.CryptoCurrency,https://www.reddit.com/r/CryptoCurrency/commen...,{},pymrq8,False,True,False,False,False,True,True,False,#0079d3,SELF-STORY,"[{'e': 'text', 't': 'SELF-STORY'}]",SELF-STORY,light,richtext,False,False,True,26,0,False,all_ads,/r/CryptoCurrency/comments/pymrq8/my_wife_star...,False,NaN,NaN,6,1633017595,1,I’m locking funds away for her for at least th...,True,False,False,CryptoCurrency,t5_2wlj3,3520483,public,self,My wife started active labor today - I started...,0,[],1.0,https://www.reddit.com/r/CryptoCurrency/commen...,all_ads,6,c301e03e-5ea6-11e8-8824-0efe9c7bcaa8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,[],False,playa-playa,,Transitioning,"[{'e': 'text', 't': ' '}]",,dark,richtext,t2_1fib37bg,False,False,False,[],False,False,1633017504,washingtonpost.com,https://www.reddit.com/r/CryptoCurrency/commen...,{},pymqrs,False,True,False,False,False,True,False,False,,CRITICAL-DISCUSSION,"[{'e': 'text', 't': '🟢 CRITICAL-DISCUSSION'}]",🟢 CRITICAL-DISCUSSION,dark,richtext,False,False,True,10,0,False,all_ads,/r/CryptoCurrency/comments/pymqrs/miamicoin_ha...,False,link,"{'enabled': False, 'images': [{'id': 'm1WtFn0A...",6,1633017516,1,,True,False,False,CryptoCurrency,t5_2wlj3,3520480,public,https://b.thumbs.redditmedia.com/9K3Aslehh1ftm...,MiamiCoin has generated $7.1 million dollars f...,0,[],1.0,https://www.washingtonpost.com/technology/2021...,all_ads,6,NaN,93.0,140.0,https://www.washingtonpost.com/technology/2021...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,[],False,Slay_the_chickens,,Warning-level2-2,"[{'e': 'text', 't': 'Tin'}]",Tin,dark,richtext,t2_3eo5j11o,False,False,False,[],False,False,1633017473,google.com,https://www.reddit.com/r/CryptoCurrency/commen...,{},pymqew,False,False,False,False,False,False,False,False,,NaN,[],NaN,dark,text,False,False,True,2,0,False,all_ads,/r/CryptoCurrency/comments/pymqew/this_best_lo...,False,NaN,NaN,6,1633017485,1,,True,False,

In [17]:
# take an exploratory look
dftest.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 79 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   all_awardings                  100 non-null    object 
 1   allow_live_comments            100 non-null    bool   
 2   author                         100 non-null    object 
 3   author_flair_background_color  75 non-null     object 
 4   author_flair_css_class         49 non-null     object 
 5   author_flair_richtext          100 non-null    object 
 6   author_flair_text              76 non-null     object 
 7   author_flair_text_color        77 non-null     object 
 8   author_flair_type              100 non-null    object 
 9   author_fullname                100 non-null    object 
 10  author_is_blocked              100 non-null    bool   
 11  author_patreon_flair           100 non-null    bool   
 12  author_premium                 100 non-null    bool

In [18]:
# take a look at the data in removed_by_category, so far 2 categories are observed : reddit, moderator
# looping up to change the timestamp for another 3 times and see if there are different categories
# observed another category : deleted, automod_filtered
dftest['removed_by_category'].value_counts()

moderator           28
reddit               6
automod_filtered     4
author               1
Name: removed_by_category, dtype: int64

In [19]:
# seeing a pattern here, seems like posts have 'removed_by_category' column
dftest[['selftext','removed_by_category', 'is_crosspostable', 
        'is_robot_indexable']][dftest['removed_by_category'].isnull()]

,selftext,removed_by_category,is_crosspostable,is_robot_indexable
0,Thought the links in this post might be useful...,NaN,True,True
1,I’m locking funds away for her for at least th...,NaN,True,True
2,,NaN,True,True
5,,NaN,True,True
6,,NaN,True,True
7,Just drop a comment saying you’re interested a...,NaN,True,True
8,I’m locking funds away for her for at least th...,NaN,True,True
9,It seems like some blockchain technologies are...,NaN,True,True
13,For the past few days it kinda seems like most...,NaN,True,True
17,"I am not a money guy, broker or really even a ...",NaN,True,True


In [20]:
dftest[['selftext','removed_by_category', 'is_crosspostable', 
        'is_robot_indexable']][dftest['removed_by_category'].notnull()]

,selftext,removed_by_category,is_crosspostable,is_robot_indexable
3,,moderator,False,False
4,[removed],moderator,False,False
10,[removed],moderator,False,False
11,[removed],moderator,False,False
12,[removed],moderator,False,False
14,,moderator,False,False
15,[removed],author,False,False
16,,automod_filtered,False,False
19,[removed],moderator,False,False
20,[removed],moderator,False,False


By and large, the attributes(columns) are similar. Will apply the same approach to dealing with filtering out the data.